In [ ]:
# setup
! pip install fire hydra-core wandb -q
! git clone --depth=1 https://github.com/martin-marek/picodo.git
! python /content/picodo/download_fineweb.py 'fineweb' 26 # 2.6B tokens

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.2/208.2 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 370.3/370.3 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.8 MB/s eta 0:00:00
Cloning into 'picodo'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 24 (delta 0), reused 16 (delta 0), pack-reused 0 (fr

In [ ]:
# imports
%cd /content/picodo
import os
from hydra import compose, initialize_config_dir
from configs import resolver_setup
from train import train_and_evaluate

/content/picodo


/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:82: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


In [3]:
# training
with initialize_config_dir(f'{os.getcwd()}/configs', version_base=None):
    c = compose(config_name='base', overrides=['+model=gpt2s', '+dataset=fw_gpt2'])
c.model.activ_dtype = 'bfloat16' # reduced precision increases arithmetic intensity
c.model.H = 256 # TPU v6e has 256x256 MXU
c.opt.batch_size = 8 # smallest batch size maximizing MFU
c.opt.peak_lr = 0.002
c.opt.b2 = 0.999 # small batch size requires high b2 (arxiv.org/abs/2507.07101)
c.opt.weight_decay = 0.02 # small batch size requires small wd
c.wandb_mode = 'online' # ['disabled', 'offline', 'online']
train_and_evaluate(c)

sharding mesh: data=1, model=1
initializing model...
n_param_nonembed=84_934_656
n_param_embed=38_597_376
n_param_actual=162_129_408
getting dataset size...
reading data...
shuffling data...
splitting data...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: martin-marek (martin-nyu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


  0%|          | 0/301591 [00:00<?, ?it/s]

eval_loss,▁
train_loss,█▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_tokens_seen,▁▁▁▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇█████
eval_loss,3.3648
n_param_actual,162129408
n_param_embed,38597376
n_param_nonembed,84934656
train_loss,3.35861
train_tokens_seen,2469666816
